In [3]:
%matplotlib inline

from PIL import Image
from matplotlib.pyplot import imshow
from torchvision import models, transforms

from torch.utils.data import Dataset, DataLoader

from torch.nn import functional as F
from torch import topk
import numpy as np
import skimage.transform
import torch
import torch.nn as nn
from dataset import PartAffordanceDataset, ToTensor, CenterCrop, Normalize
import pandas as pd
import random
import sys
import tqdm
import yaml

from addict import Dict

In [10]:
CONFIG = Dict(yaml.safe_load(open('./result/config.yaml')))

In [13]:
data = PartAffordanceDataset(CONFIG.labeled_data,
                           config=CONFIG,
                           transform=transforms.Compose([
                               CenterCrop(CONFIG),
                               ToTensor(),
                               Normalize()
                           ]))

In [17]:
train_loader = DataLoader(train_data, batch_size=1, shuffle=True, num_workers=CONFIG.num_workers)

In [34]:
model = models.vgg16_bn(pretrained=False)
model.classifier[6] = nn.Linear(in_features=4096, out_features=7, bias=True)

In [35]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [1]:
from model import VGG16

In [2]:
model = VGG16(3, 100)